Created by: Kenneth Guillont

# Introduction

This is notebook utilizes Py2neo to load a Neo4j Database to create a graphical representation of our data and their relationships to one another. Our data is contained in the respective files: nodes.tsv and edges.tsv. Nodes are entities of our dataset and the edges are the relationships that join entities together.

# Loading the Neo4j

### Importing needed libraries

- The `pandas` library will only be used to read our files. Our data is stored in tsv files, or tab seperated values.
- The `py2neo` Driver is used to access the Neo4j database by using uri and user creditials. Futhermore, this driver is able to preform operations on the database.
- The `json` library is used to convert our files from tsv to json, which will increase the rate of loading.

In [27]:
import pandas as pd
from py2neo import Graph, Node, Relationship
import json

### Connecting to Neo4j Instnace

Connection creditials are used to connect this notebook to the Neo4j instance. This includes the URI, Username, and Password.

In [21]:
# Connect to the Neo4j instance
graph = Graph("bolt://localhost:7690", auth=("neo4j", "password"))

### Reading the .tsv files

Reads nodes.tsv and edges.tsv, then stores each in nodes and edges respectively. The separators for each dataset are tabs, `sep=\t` is included in our call to the function `read_csv()` to accurately secion our data into their respective columns.

In [22]:
# Load the nodes and edges CSV files
nodes = pd.read_csv('/Users/kennethguillont/Desktop/Hunter/fall2024/CSCI49375/hetionet/Data/nodes.tsv', sep='\t')
edges = pd.read_csv('/Users/kennethguillont/Desktop/Hunter/fall2024/CSCI49375/hetionet/Data/edges.tsv', sep='\t')

### Convert .tsv to .json

Converting to a .json file prior to loading the data appearing to improve the overall time it took to load the database.

In [15]:
# Convert nodes to JSON
nodes_json = nodes.to_json(orient='records', lines=True)
with open('nodes.json', 'w') as f:
    f.write(nodes_json)

# Convert edges to JSON
edges_json = edges.to_json(orient='records', lines=True)
with open('edges.json', 'w') as f:
    f.write(edges_json)

print("JSON files created: nodes.json and edges.json")

JSON files created: nodes.json and edges.json


### Loading Nodes and Edges

In both cases, a for-loop is implemented to iterate through each file. An additional step is needed for loading the edges from a Json file because the IDs of both the sourse and target nodes must be stripped by their non-alphanumeric characters to obtain the `kind` and `id`.

Additional cells were implemented to provide verification to ensure loading was successful.

In [ ]:
# Load nodes from JSON
with open('nodes.json', 'r') as f:
    for line in f:
        node_data = json.loads(line)
        node = Node(node_data['kind'], id=node_data['id'], name=node_data['name'], kind=node_data['kind'])
        graph.merge(node, node_data['kind'], "id")

In [ ]:
# Verify data, just to make sure it actually loaded
nodesCreated = graph.run("MATCH (n) RETURN n LIMIT 10").data()
print(nodesCreated)

In [ ]:
# Load edges from JSON
with open('edges.json', 'r') as f:
    for line in f:
        edge_data = json.loads(line)

        # Match source and target nodes based on their IDs
        source_node = graph.nodes.match(kind=edge_data['source'].split("::")[0], id=edge_data['source']).first()
        target_node = graph.nodes.match(kind=edge_data['target'].split("::")[0], id=edge_data['target']).first()

        # Create the relationship if both nodes exist
        if source_node and target_node:
            relationship = Relationship(source_node, edge_data['metaedge'], target_node)
            graph.create(relationship)

print("Data imported successfully. Finally!")


In [11]:
# Verify data, just to make sure it actually loaded
relationshipsCreated = graph.run("MATCH (a)-[r]->(b) RETURN a, r, b LIMIT 10").data()
print(relationshipsCreated)

[{'a': Node('Anatomy', id='Anatomy::UBERON:0000002', kind='Anatomy', name='uterine cervix'), 'r': AeG(Node('Anatomy', id='Anatomy::UBERON:0000002', kind='Anatomy', name='uterine cervix'), Node('Gene', id='Gene::1000', kind='Gene', name='CDH2')), 'b': Node('Gene', id='Gene::1000', kind='Gene', name='CDH2')}, {'a': Node('Anatomy', id='Anatomy::UBERON:0000002', kind='Anatomy', name='uterine cervix'), 'r': AeG(Node('Anatomy', id='Anatomy::UBERON:0000002', kind='Anatomy', name='uterine cervix'), Node('Gene', id='Gene::10001', kind='Gene', name='MED6')), 'b': Node('Gene', id='Gene::10001', kind='Gene', name='MED6')}, {'a': Node('Anatomy', id='Anatomy::UBERON:0000002', kind='Anatomy', name='uterine cervix'), 'r': AeG(Node('Anatomy', id='Anatomy::UBERON:0000002', kind='Anatomy', name='uterine cervix'), Node('Gene', id='Gene::10004', kind='Gene', name='NAALADL1')), 'b': Node('Gene', id='Gene::10004', kind='Gene', name='NAALADL1')}, {'a': Node('Anatomy', id='Anatomy::UBERON:0000002', kind='Anato

# Qeuries

**1. Given a disease id, what is its name, what are drug names that can treat or palliate this disease, what are gene names that cause this disease, and where this disease occurs? Obtain and output this information in a single query.**

In [26]:
query1 = f"""
    MATCH (d:Disease {{id: '{'Disease::DOID:1324'}'}})
    MATCH (d)-[:DaG]->(g:Gene)  // Disease associates with Gene
    MATCH (a:Anatomy)<-[:DlA]-(d)  // Anatomy localizes Disease
    MATCH (c:Compound)-[:CtD|CpD]->(d)  //treat or palliate this disease
    RETURN d.name AS disease_name, 
           COLLECT(DISTINCT c.name) AS drugs, 
           COLLECT(DISTINCT g.name) AS gene_names, 
           COLLECT(DISTINCT a.name) AS locations
    """

graph.run(query1)

disease_name | drugs                                                                                                                                                                                                                                        | gene_names                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | locations                                                                                                                                                                                                                                                                                                                                                                                                                                                 
--------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

For graphical representation, I used this cypher Query:

MATCH (d:Disease {id: 'Disease::DOID:10763'}) <br>
MATCH (d)-[:DlA]->(a:Anatomy) <br>
MATCH (a)-[:AuG]->(g:Gene) <br>
MATCH (c:Compound)-[:CdG]->(g) <br>
WHERE NOT (c)-[:CtD]->(d) AND NOT (c)-[:CpD]->(d) <br>
RETURN d, a, g, c

![Query 1](./Images/query1.png)


**2. We assume that a compound can treat a disease if the compound up-regulates/down-regulates a gene, but the location down-regulates/up-regulates the gene in an opposite direction where the disease occurs. Find all compounds that can treat a new disease (i.e. the missing edges between compound and disease excluding existing drugs). Obtain and output all drugs in a single query.**

In [29]:
query2 = f"""MATCH (d:Disease {{id: '{'Disease::DOID:1324'}'}})
MATCH (d)-[:DlA]->(a:Anatomy)
MATCH (a)-[:AuG]->(g:Gene)
MATCH (c:Compound)-[:CdG]->(g) 
WHERE NOT (c)-[:CtD]->(d) AND NOT (c)-[:CpD]->(d) 
RETURN COLLECT(DISTINCT c.name) AS new_treatment_compounds"""

graph.run(query2)

new_treatment_compounds


For graphical representation, I used this cypher Query:

MATCH (d:Disease {id: 'Disease::DOID:1324'})   <br>
MATCH (d)-[:DlA]->(a:Anatomy)   <br>
MATCH (a)-[:AuG]->(g:Gene)   <br>
MATCH (c:Compound)-[:CdG]->(g)   <br>
WHERE NOT (c)-[:CtD]->(d) AND NOT (c)-[:CpD]->(d)  <br>
RETURN d, a, g, c

![Alt text](./Images/query2.png)


# Areas of Improvement

**Data Loading Efficiency:** One of the primary areas for improvement is the time required to load the database, which took over 10 hours to iterate through the 23K nodes and 1.3 million edges. Optimizing the algorithm for traversing the graph or implementing a batching approach could significantly reduce the load time, enhancing overall efficiency.

**User Interface and Output Presentation:** The current output formatting could be improved to provide a cleaner, more user-friendly experience. Enhancing the presentation of results—while maintaining the time efficiency of the underlying code—would make the output easier to interpret, improving both usability and clarity for users.

# Python Command Line Client

Please see the `neo4jCLC.py` file for the implementation of the Command Line Client.